## Eigenvector Metal Etch Data for Fault Detection Evaluation
Jaganadh Gopinadhan (Jagan)

https://www.linkedin.com/in/jaganadhg/

### Introduction

This Data Set is one of the oldest Microfabrication/Semiconductor process-related open data. The data was released by Eigenvector, a niche industry analytics company. The information was used in a research study and paper published in 1999[1]. The first author Barry M. Wise is one of the company's founding members.

The data was taken from LAM 9600 Metal Etcher[2]. Etching is used in microfabrication to chemically remove layers from the surface of a wafer during manufacturing[3]. The data was released in Matlab format, suitable for analyzing using the PLS Toolbox for Eigenvector. There are three .mat files; MACHINE_Data.mat, OES_DATA.mat, and RFM_DATA.mat. A detailed note on the data and attributes available in reference [1] and [2]. Since the data is in Matlab format, we created a python based parser to convert the data in pandas DataFrames. This parser should enable Data Miners and Data Scientists to play around with Open Source tools like Python or R. 

### The Eigenvector Etch Data Parser

The Eigenvector Etch Data Parse is developed to read Matlab data files. The parse reads each file and converts the calibration data (sensor data) and test data (sensor data) into a single DataFrame. The parse introduced an additional field in the data  'fault_name', which helps the user identify the normal/calibration wafers and test wafers(with defects). We tested the parser in Python3 environments only; if you are looking for Python2 compatibility, please test and create a bug/pull request as applicable.  The source code is released under Apache 2.0 license and is available at https://github.com/jaganadhg/egvsemicon. 

### Etch Data

The Eigenvector Etch Data is provided in three Matlab vector files MACHINE_Data.mat, OES_DATA.mat, and RFM_DATA.mat. The MACHINE_Data.mat file consists of the engineering variables, time, and the etch recipe steps. The variable/feature categories are pressure, gas flow rate, and power (If you wonder why gas flow is here, it is part of the semiconductor chemistry process and a more significant topic beyond the note!). The OES_DATA.mat file consists of the optical emission spectroscopy (OES) of the plasma. OES description is available in Hitachi's reference pge[4]. The file RFM_DATA.mat contains radio-frequency monitoring (RFM) system to monitor the power and phase relationships of the plasma generator. 

The OES data do not come with sensor names, unlike the Machine Data/Engineering Variable and RFM Data. The data is a field wave_axis which represents wavelengths in nm of peaks. 

The RFM Data has associated information related to the unit of variables (units: [ 71x6  char]  Units of the variables). This project provides the variable and its unit mapping as a CSV file. There are 35 variables measured with VRMS [5], 20 variables with the degree, 15 variables with IRMS [5], and one variable with the second. The pandas DataFrame from the parser will not be providing this information. The sensor names are masked to protect the process and machine details in this data. It is difficult to determine the sensor name without explicit Etch machine knowledge. However, from the units, a smart and experienced Process Engineer would infer some. 


### Reference

[1] B.M. Wise, N.B. Gallagher, S.W. Butler, D.D. White, Jr. and G.G. Barna, “A Comparison of Principal Components Analysis, Multi-way Principal Components Analysis, Tri-linear Decomposition and Parallel Factor Analysis for Fault Detection in a Semiconductor Etch Process”, J. Chemometrics, 13, 379­396 (1999)

[2] https://eigenvector.com/resources/data-sets/

[3] https://en.wikipedia.org/wiki/Etching_(microfabrication) 

[4] https://hha.hitachi-hightech.com/en/blogs-events/blogs/2017/10/25/optical-emission-spectroscopy-(oes)/

[5] IRMS - The current in an alternating current circuit varies continuously in direction and magnitude. ... Diagrams denote this current as "IRMS," with the "RMS" in subscript. A constant level of the root means square current dissipates the same amount of heat through a resistor as the alternating current does.
VRMS - In electricity: Alternating-current circuits. The root-mean-square (RMS) voltage of a sinusoidal source of electromotive force (Vrms) is used to characterize the source. It is the square root of the time average of the voltage squared. - https://www.quora.com/What-are-VRMS-and-IRMS 